# Trabalho prático Visão por Computador

## Group 13  <br>
Nelson José Marques Martins Almeida pg52697 <br> 
Carlos Gustavo Silva Pereira pg53723

## Enunciado do trabalho 1 - grupo (2-3 elementos)
Neste trabalho pretende-se explorar modelos de deep learning aplicados ao dataset GTSRB (dataset alemão de sinais de trânsito) para obter o melhor resultado possível em termos de accuracy no dataset de teste (o melhor resultado publicado é 99.82%). <br>
O trabalho divide-se em duas partes:
Numa primeira parte é suposto treinar modelos aplicando "data augmentation", tanto em pré-processamento como dinâmico. Pretende-se nesta primeira parte explorar os filtros e métodos de processamento de imagem e avaliar o impacto dos mesmos no desempenho final da rede. Parte da avaliação será ditada pelo uso das técnicas de processamento de imagem utilizadas.  Devem ser treinadas vários modelos, com "data augmentation" diferentes, e ser realizada uma análise aos resultados obtidos.
Numa segunda fase é pretendido estudar o potencial de utilizar ensembles de redes. As redes que farão parte do ensemble são as treinadas na primeira fase. 
Dataset alemão:
Imagens para treino (ZIP)https://www.di.uminho.pt/~arf/storage/dl/train_images.zip
Imagens para teste (ZIP)https://www.di.uminho.pt/~arf/storage/dl/test_images.zip
Para além do notebook (ou scripts python avulsas) pretende-se um pequeno relatório que descreva as opções tomadas e apresente os testes realizados. O notebook ou as scripts também devem ser devidamente comentadas.

In [4]:
# Lets import the needed dependencies
import torch
import torchvision
from torchvision.transforms import v2

import torchinfo

import matplotlib.pyplot as plt
import numpy as np
import time

# importing a module with utilities for displaying stats and data
import sys
sys.path.append('./util')
import vcpi_util
from SyntheticDataGeneration import *
import os

from tqdm import tqdm
import requests

In [6]:
# Lets download the needed images if they are not already downloaded
if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists("data/train"):
    print("Downloading train images...")
    requests.get("https://www.di.uminho.pt/~arf/storage/dl/train_images.zip")
    os.system("unzip train_images.z1ip -d data/train")
    os.system("rm train_images.zip")

if not os.path.exists("data/test"):
    print("Downloading test images...")
    requests.get("https://www.di.uminho.pt/~arf/storage/dl/test_images.zip")
    os.system("unzip test_images.zip -d data/test")
    os.system("rm test_images.zip")

print("Data downloaded and extracted successfully!")

Data downloaded and extracted successfully!


In [ ]:
#Lets 